# 📍 NutriChef AI - Phase 2: Gradio Frontend App
This notebook creates the frontend UI for NutriChef AI using Gradio.
Users can upload a fridge photo or record their voice to get meal plans, recipes, nutrition facts, and health advice.

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

## 📍Step 1: Install and Import Basic Libraries

In [ ]:
# Move to project folder (only if you're on Colab and this path is correct)
%cd /content/drive/MyDrive/Colab Notebooks/NutriChefAI

# Whisper (speech-to-text from audio/video)
!pip install git+https://github.com/openai/whisper.git

# YouTube downloader
!pip install yt-dlp

# LangChain ecosystem + LangSmith for tracing
!pip install langchain langchain-community langchain-openai langsmith

# Core model libraries
!pip install openai huggingface_hub transformers

# Vector database + embedding models
!pip install chromadb sentence-transformers

# Gradio for app UI
!pip install gradio

### Import Needed Libraries

In [ ]:
import os

# Set LangSmith credentials
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""
os.environ["LANGCHAIN_PROJECT"] = "NutriChefAI"

In [ ]:
# from langsmith import Client

# client = Client()
# project = client.read_project(project_name="NutriChefAI")
# print(f"✅ LangSmith connected to project: {project.name}")

In [ ]:
# === Imports ===
import gradio as gr
import os
import sys
import random
import json
import yt_dlp
import whisper
import ast
import re
import chromadb
import pandas as pd
# from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from chromadb.config import Settings
from chromadb import PersistentClient
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langsmith import traceable
from tqdm import tqdm

from langchain.tools import Tool

from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Load the cleaned recipe dataset with nutrition column
recipes_df = pd.read_csv("./database/recipes.csv")
# Rename if needed
recipes_df["recipe_name"] = recipes_df["recipe_name"].str.lower().str.strip()
recipes_df = recipes_df.rename(columns={"recipe_name": "name"})

#++++++++++++++++++++++++++++++

# Load RAW_recipes (limit to useful columns)
raw_df = pd.read_csv("./database/RAW_recipes.csv", usecols=[
    "name", "minutes", "nutrition", "steps", "description", "ingredients"
])

# Standardize text fields
raw_df["name"] = raw_df["name"].str.lower().str.strip()
raw_df["description"] = raw_df["description"].fillna("")
raw_df["steps"] = raw_df["steps"].apply(ast.literal_eval).apply(lambda x: " ".join(x) if isinstance(x, list) else "")
raw_df["ingredients"] = raw_df["ingredients"].apply(ast.literal_eval).apply(lambda x: ", ".join(x) if isinstance(x, list) else "")

# Convert nutrition to string for safety if needed
raw_df["nutrition"] = raw_df["nutrition"].apply(lambda x: str(x) if isinstance(x, list) else x)

# Add empty columns to raw_df to match existing schema if needed
raw_df["primaryCategories"] = "general"
raw_df["cuisine_path"] = "unknown"
raw_df["timing"] = raw_df["minutes"].astype(str) + " minutes"

combined_df = pd.concat([recipes_df, raw_df], ignore_index=True)

#+++++++++++++++++++++++++++++++

# Add models/utils folder to path if needed
sys.path.append('./')

# Import backend modules
from models.vision_model import IngredientDetector
# from models.speech_to_text import SpeechToText
from models.meal_generator import create_daily_meal_plan
from models.food_classifier import FoodClassifier
# from utils.recipe_retriever import embed_query, search_recipes
# from utils.nutrition_generator import generate_nutrition_facts_and_advice


# For vector database and OpenAI

from openai import OpenAI
from PIL import Image
os.makedirs("./downloads", exist_ok=True)
os.environ["OPENAI_API_KEY"] = ""
# from huggingface_hub import login
# login(token="")

### Setup Clients and Models Once

In [ ]:
class SpeechToText:
    def __init__(self):
        import whisper
        self.model = whisper.load_model("base")

    def extract_ingredients(self, audio_file_path):
        try:
            result = self.model.transcribe(audio_file_path)
            text = result["text"]
            ingredients = [x.strip() for x in text.split(",") if x.strip()]
            return ingredients
        except Exception as e:
            print(f"🔥 Error transcribing audio: {e}")
            return []

In [ ]:
# rm -rf ./database/vector_store

In [ ]:
from chromadb.errors import NotFoundError
# === Setup backend models ===

# Vision model (Image captioning)
vision_detector = IngredientDetector()

# Whisper model (Speech to Text)
speech_detector = SpeechToText()

# Embedding model
# embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

#+++++++++++++++++++++++++++++++++
# Clear and re-add all data
docs = combined_df["description"].fillna("") + ". " + combined_df["steps"].fillna("")
metadatas = combined_df[["name", "ingredients", "timing", "cuisine_path"]].to_dict(orient="records")

#+++++++++++++++++++++++++++++++++
# Vector store (ChromaDB)
# chroma_client = chromadb.PersistentClient(path="./database/vector_store")
# chroma_client = PersistentClient(path="./database/vector_store")

# collection = chroma_client.get_or_create_collection(name="recipes") ***

#++++++++++++++++++++++++++++++++
# Re-create the collection from scratch
# Initialize persistent client
chroma_client = PersistentClient(path="./database/vector_store")

# Try deleting only if exists
try:
    chroma_client.delete_collection("recipes")
    print("🗑️ Old 'recipes' collection deleted.")
except NotFoundError:
    print("⚠️ No existing 'recipes' collection found. Skipping deletion.")

collection = chroma_client.get_or_create_collection("recipes")

def batched_add_to_chroma(collection, docs, embeddings, metadatas, batch_size=5000):
    for i in range(0, len(docs), batch_size):
        batch_docs = docs[i:i+batch_size]
        batch_embeddings = embeddings[i:i+batch_size]
        batch_metadatas = metadatas[i:i+batch_size]
        batch_ids = [f"recipe_{j}" for j in range(i, i+len(batch_docs))]

        collection.add(
            documents=batch_docs,
            embeddings=batch_embeddings,
            metadatas=batch_metadatas,
            ids=batch_ids
        )

# ✅ No need to clear manually — collection was just deleted

# Embed everything first
all_docs = docs.tolist()
all_embeddings = embedding_model.embed_documents(all_docs)
all_metadatas = metadatas

# Add in batches
batched_add_to_chroma(collection, all_docs, all_embeddings, all_metadatas)

#++++++++++++++++++++++++++++++++


# Connect to a persistent Chroma client

video_collection = chroma_client.get_or_create_collection(name="youtube_videos")

# OpenAI Client
client = OpenAI()

# Load Whisper model
whisper_model = whisper.load_model("base")

food_detector = FoodClassifier()


In [ ]:

# 1. Prompt template

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are NutriChef AI, a professional and friendly culinary assistant.

- You always remember the user's recent questions and give clear, complete, and helpful answers.
- If a user follows up with something like 'is it healthy?' or 'how to make it?', assume it's about the last discussed dish.
- Respond confidently. Do not ask for clarification unless absolutely necessary.
- Avoid repeating or restating the user’s question in replies.
"""),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# 🔁 Chain with memory
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)
memory_chain = RunnableWithMessageHistory(
    chat_prompt | chat_model,
    lambda session_id: InMemoryChatMessageHistory(),
    input_messages_key="input",
    history_messages_key="history"
)

# Gradio States (for memory)
ingredients_state = gr.State("")
meal_plan_state = gr.State({})

In [ ]:
import uuid

def get_session_id():
    return str(uuid.uuid4())

session_id = gr.State(get_session_id())
chat_history = gr.State([])

## 📍Step 2: Write Backend Helper Functions

In [ ]:

def extract_dish_name(user_message: str) -> str:
    """
    Extracts a likely dish or food item from the user message using general NLP rules.
    """
    user_message = user_message.lower()

    # Explicit patterns
    patterns = [
        r"(?:how to make|recipe for|cook|prepare|want(?: to try)?|suggest(?: me)?|make)\s+([a-zA-Z\s]+)",
        r"(?:i(?:'d)? like|give me|something like)\s+([a-zA-Z\s]+)",
    ]

    for pattern in patterns:
        match = re.search(pattern, user_message)
        if match:
            candidate = match.group(1).strip()
            candidate = re.sub(r"\b(for dinner|for lunch|please|recipe)?\b", "", candidate).strip()
            if 1 <= len(candidate.split()) <= 6:
                return candidate

    # Fallback: last 2–4 words as a potential noun phrase
    tokens = user_message.split()
    if len(tokens) >= 2:
        fallback = " ".join(tokens[-4:])  # last 4 words
        return fallback.strip()

    return ""

def detect_dish_with_llm(message):
    prompt = f"""
You're a food assistant. Extract the specific dish or food name the user is referring to in this message:

"{message}"

If no dish is mentioned, return "none".
Only return the dish name.
"""
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

In [ ]:
#--------------------------------------------------------
#       🧼 Preprocessing / Input Handling
#--------------------------------------------------------

def resize_image(image, size=(384, 384)):
    """
    Safely resize uploaded image for vision models.
    """
    return image.resize(size)
#_______________________

@traceable(name="ProcessImagePipeline")
def process_image(image, mode="Generate New Meal Plan"):
    """
    Handles image > ingredients > generate meal OR search recipes based on user mode.
    """
    try:
        if image is None:
            return "❌ Please upload a valid image."

        print("📸 Processing image...")
        image = resize_image(image)

        ingredients, caption = vision_detector.detect_ingredients(image)
        print(f"✅ Detected Ingredients: {ingredients}")

        if not ingredients:
            return "❌ No ingredients could be detected from the image."

        if mode == "Generate New Meal Plan":
            meal_plan = generate_meal_plan(ingredients)
            nutrition = generate_nutrition_facts_and_advice(meal_plan)
            return format_output(ingredients, meal_plan, nutrition)

        elif mode == "Search Existing Recipes":
            ingredient_query = ", ".join(ingredients)
            recipes = search_recipes(ingredient_query)
            return f"""🧺 **Detected Ingredients**:
{ingredient_query}

📚 **Top Recipes**:
""" + "\n\n".join(recipes)

        else:
            return "❌ Invalid mode selected."

    except Exception as e:
        print(f"🔥 Error in process_image(): {e}")
        return "❌ An unexpected error occurred."



# === Vision Model for Ingredient Detection ===

class IngredientDetector:
    """
    Loads a BLIP model for ingredient detection from images.
    """
    def __init__(self, device=None):
        import torch
        from transformers import BlipProcessor, BlipForConditionalGeneration

        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")
        self.processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
        self.model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(self.device)

    def detect_ingredients(self, image):
        """
        Detects ingredients and generates a caption from the image.
        """
        try:
            inputs = self.processor(image, return_tensors="pt").to(self.device)
            output = self.model.generate(**inputs)
            caption = self.processor.decode(output[0], skip_special_tokens=True)

            # Extract ingredients from caption simply (split commas)
            ingredients_list = [x.strip() for x in caption.split(',') if x.strip()]
            return ingredients_list, caption
        except Exception as e:
            print(f"🔥 Error detecting ingredients: {e}")
            return [], "Error"



#_______________________

@traceable(name="ExtractIngredientsFromAudio")
def extract_ingredients(audio_path):
    """
    Transcribes the audio and extracts ingredients using GPT.
    """
    try:
        print(f"🎤 Transcribing audio from: {audio_path}")
        result = whisper_model.transcribe(audio_path)
        full_text = result["text"]
        print(f"📝 Transcription: {full_text}")

        # Extract ingredients using GPT
        prompt = f"""
You're a kitchen assistant. The user said:

"{full_text}"

From this sentence, extract and return only the list of ingredients mentioned (just food items).
Return them as a Python list. Do not include extra commentary.
"""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You extract ingredients from text."},
                {"role": "user", "content": prompt}
            ]
        )

        extracted = response.choices[0].message.content.strip()
        print(f"✅ Extracted: {extracted}")

        # Evaluate string safely as list (e.g., "['eggs', 'tomatoes']")
        ingredients = eval(extracted)
        return ingredients

    except Exception as e:
        print(f"🔥 Error extracting ingredients: {e}")
        return []

#_______________________

@traceable(name="ProcessAudioPipeline")
def process_audio(audio_path, mode="Generate New Meal Plan"):
    try:
        if not audio_path:
            print("❌ No audio path received.")
            return "❌ Please upload a valid audio file."

        print("🎤 Starting to process audio...")
        ingredients = speech_detector.extract_ingredients(audio_path)
        print(f"✅ Detected Ingredients: {ingredients}")

        if not ingredients:
            print("❌ No ingredients detected.")
            return "❌ No ingredients could be detected from the audio."

        print(f"🔄 Selected Mode: {mode}")

        if mode == "Generate New Meal Plan":
            meal_plan = generate_meal_plan(ingredients)
            print(f"✅ Generated Meal Plan.")
            nutrition = generate_nutrition_facts_and_advice(meal_plan)
            print(f"✅ Generated Nutrition Advice.")
            output = format_output(ingredients, meal_plan, nutrition)
            print(f"✅ Final Output Ready.")
            return output

        elif mode == "Search Existing Recipes":
            ingredient_query = ", ".join(ingredients)
            recipes = search_recipes(ingredient_query)
            print(f"✅ Retrieved {len(recipes)} Recipes.")
            return f"""🧺 **Detected Ingredients**:
{ingredient_query}

📚 **Top Recipes**:
""" + "\n\n".join(recipes)

        else: # elif mode == "invalid mode"
            print("❌ Invalid mode selected.")
            return "❌ Invalid mode selected."

    except Exception as e:
        print(f"🔥 FULL ERROR TRACE: {e}")
        print(f"The audio path type is: {type(audio_path)} and the mode is: {mode}")
        # return "❌ An unexpected error occurred."
        return f"The audio path type is: {type(audio_path)} and the mode is: {mode}"

#__________________________________________________________________________________________________________________________________________________

#-----------------------------------------------------------
#         🍳 Meal & Nutrition Generation
#-----------------------------------------------------------

@traceable(name="GenerateDetailedMealPlan")
def generate_meal_plan(ingredients_list):
    try:
        ingredients_query = ", ".join(ingredients_list)

        prompt = f"""
You are an AI chef. Given the following available ingredients: {ingredients_query},
suggest one meal for breakfast, one for lunch, and one for dinner.

For each meal, provide:
- A title
- A short list of ingredients (from what's available)
- Simple step-by-step instructions on how to make it

Use this exact format:

Breakfast:
Title: ...
Ingredients: ...
Instructions: ...

Lunch:
Title: ...
Ingredients: ...
Instructions: ...

Dinner:
Title: ...
Ingredients: ...
Instructions: ...
"""

        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a professional recipe chef and meal planner."},
                {"role": "user", "content": prompt}
            ]
        )

        return response.choices[0].message.content

    except Exception as e:
        print(f"🔥 Error generating detailed meal plan: {e}")
        return "❌ Error generating detailed meal plan."

#_______________________

def lookup_nutrition(meal_plan, recipes_df):
    """
    Tries to match each meal in the plan to a recipe and extracts nutrition info.
    Returns a summary string.
    """
    results = []
    for line in meal_plan.lower().splitlines():
        if ":" not in line:  # Skip lines without clear meal info
            continue

        label, meal = line.split(":", 1)
        meal = meal.strip()

        match = recipes_df[recipes_df["name"].str.contains(meal, case=False, na=False)]
        if not match.empty:
            nutrition = match.iloc[0]["nutrition"]
            try:
                nutrition_data = ast.literal_eval(nutrition)
                nutrition_summary = f"Calories: {nutrition_data[0]} kcal, Fat: {nutrition_data[1]}g, Carbs: {nutrition_data[2]}g, Protein: {nutrition_data[3]}g"
            except Exception:
                nutrition_summary = nutrition  # fallback if parsing fails
        else:
            nutrition_summary = "Nutrition data not found."

        results.append(f"{label.capitalize()}: {meal} ➤ {nutrition_summary}")

    return "\n".join(results)

#_______________________

def get_nutrition_info(meal_name: str) -> str:
    """
    Look up nutrition info for a given meal name using the dataset.
    """
    match = recipes_df[recipes_df["name"].str.contains(meal_name, case=False, na=False)]
    if not match.empty:
        nutrition = match.iloc[0]["nutrition"]
        try:
            nutrition_data = ast.literal_eval(nutrition)
            return f"Calories: {nutrition_data[0]} kcal, Fat: {nutrition_data[1]}g, Carbs: {nutrition_data[2]}g, Protein: {nutrition_data[3]}g"
        except:
            return nutrition  # fallback
    else:
        return "Nutrition data not found."

from langchain.tools import Tool

nutrition_tool = Tool(
    name="get_nutrition_info",
    func=get_nutrition_info,
    description="Use this tool to fetch nutrition data for a meal name. Input is the meal name as a string."
)
#_______________________

@traceable(name="GenerateNutritionAdvice")
def generate_nutrition_facts_and_advice(meal_plan):
    try:
        print("🧪 Meal Plan Input:")
        print(meal_plan)

        # Step 1: Lookup Nutrition from Dataset
        grounded_nutrition = lookup_nutrition(meal_plan, recipes_df)
        print("📊 Grounded Nutrition Extracted:")
        print(grounded_nutrition)

        # Step 2: Prepare prompt
        prompt = f"""
You are a nutritionist.

Here is the meal plan for today:
{meal_plan}

And here are the grounded nutrition facts for each meal:
{grounded_nutrition}

Now:
- Give a short nutritional summary for the day
- Suggest 2 health tips to improve the meal plan
"""

        # Step 3: Run OpenAI GPT
        print("🧠 Sending to OpenAI...")
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a licensed dietitian."},
                {"role": "user", "content": prompt}
            ]
        )

        print("✅ GPT Response Received.")
        return response.choices[0].message.content

    except Exception as e:
        print(f"🔥 ERROR in generate_nutrition_facts_and_advice(): {e}")
        return "❌ Error generating nutrition advice."

#_______________________

@traceable(name="FormatFinalOutput")
def format_output(ingredients_list, meal_plan, nutrition_advice):
    """
    Combines all outputs (ingredients, meals, advice) into a clean text output.
    """
    try:
        final_text = f"""🧺 **Detected Ingredients**:
{', '.join(ingredients_list)}

🍽️ **Meal Plan**:
{meal_plan}

🧪 **Nutrition Facts and Advice**:
{nutrition_advice}
"""
        return final_text

    except Exception as e:
        print(f"🔥 Error formatting output: {e}")
        return "❌ Error formatting final output."

#________________________________________________________________________________________________________________________________________

#------------------------------------------------------------
#            🔍 Embedding + Retrieval
#------------------------------------------------------------

@traceable(name="EmbedQuery")
def embed_query(text_query):
    return embedding_model.embed_query(text_query)

#_______________________

@traceable(name="SearchRecipes")
def search_recipes(ingredient_query, top_k=5):
    query_embedding = embed_query(ingredient_query)
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        include=["documents"]
    )
    return [f"• {r}" for r in results["documents"][0]]

#_______________________

@traceable(name="GetRecipeContext")
def get_recipe_context(ingredients_list, top_k=3):
    ingredient_query = ", ".join(ingredients_list)
    query_embedding = embedding_model.embed_query(ingredient_query)

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=top_k,
        include=["documents", "metadatas"]
    )

    context_lines = []
    for i, doc in enumerate(results["documents"][0]):
        meta = results["metadatas"][0][i]
        cuisine = meta.get("cuisine_path", "Unknown")
        timing = meta.get("timing", "N/A")
        instructions = meta.get("instructions", "No directions available.")
        context_lines.append(
            f"• {doc} ({cuisine}, takes {timing})\nDirections: {instructions}"
        )

    return "\n\n".join(context_lines)

#____________________________________________________________________________________________________________________________________

# Tool 1: Recipe Search
recipe_tool = Tool(
    name="SearchRecipe",
    func=search_recipes,
    description="Search for recipes based on a list of ingredients."
)

# Tool 2: Nutrition Info Lookup
nutrition_tool = Tool(
    name="GetNutritionInfo",
    func=get_nutrition_info,
    description="Get nutrition details for a specific meal name."
)

# Tool 3: Generate Meal Plan
meal_plan_tool = Tool(
    name="GenerateMealPlan",
    func=generate_meal_plan,
    description="Generate a daily meal plan based on a list of ingredients."
)
#___________________
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.3)

agent = initialize_agent(
    tools=[recipe_tool, nutrition_tool, meal_plan_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

#____________________________________________________________________________________________________________________________________


#--------------------------------------------------------
#              💬 Assistant Chatbot
#--------------------------------------------------------

def get_context_from_query(query):
    try:
        results = collection.query(
            query_embeddings=embedding_model.embed_query(query),
            n_results=3,
            include=["documents", "metadatas"]
        )
        contexts = []
        for doc, meta in zip(results["documents"][0], results["metadatas"][0]):
            ctx = f"{doc} | Cuisine: {meta.get('cuisine_path', 'Unknown')}, Time: {meta.get('timing', 'N/A')}, Instructions: {meta.get('instructions', 'N/A')}"
            contexts.append(ctx)
        return "\n\n".join(contexts)
    except Exception as e:
        print(f"❌ Error in get_context_from_query(): {e}")
        return "No relevant recipes found."


@traceable(name="SmartNutriChefChat")
def handle_chat(user_message, session_id, chat_history, last_dish):
    try:
        if not user_message.strip():
            return gr.update(value=chat_history), "", last_dish

        print(f"🧠 User input: {user_message}")

        # Handle "how to make it" follow-up
        if "how to make" in user_message.lower() and "it" in user_message.lower() and last_dish:
            user_message = f"How do I make {last_dish}?"

        # Get recipe context for retrieval
        recipe_context = get_context_from_query(user_message)

        # Generate response using memory-aware chain
        response = memory_chain.invoke(
            {"input": user_message, "recipe_context": recipe_context},
            config={"configurable": {"session_id": session_id}}
        )

        # Extract new potential dish name
        import re
        match = re.search(r"(?:want|try|make|suggest(?: me)?|recipe for|dish like)\s+([a-zA-Z\s]+)", user_message.lower())
        if match:
            candidate = match.group(1).strip()
            if len(candidate.split()) <= 6:  # avoid noisy or vague phrases
                last_dish = candidate

        # Update chat
        chat_history.append([user_message, response.content])
        return gr.update(value=chat_history), "", last_dish

    except Exception as e:
        print(f"🔥 Error in handle_chat: {e}")
        return gr.update(value=chat_history), "", last_dish

#________________________________________________________________________________________________________________________________

#----------------------------------------------------
#          🎥 YouTube QA Pipeline
#----------------------------------------------------

@traceable(name="DownloadYouTubeAudio")
def download_youtube_audio(youtube_url, output_path="./downloads"):
    """
    Downloads audio from a YouTube video and saves as an MP3 file.
    """
    try:
        ydl_opts = {
            'format': 'bestaudio/best',
            'outtmpl': f'{output_path}/%(title)s.%(ext)s',
            'postprocessors': [{
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }],
            'quiet': False
        }

        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            info = ydl.extract_info(youtube_url, download=True)
            filename = ydl.prepare_filename(info)
            filename = filename.replace('.webm', '.mp3').replace('.m4a', '.mp3')

        print(f"✅ Downloaded audio file: {filename}")
        return filename

    except Exception as e:
        print(f"🔥 Error downloading YouTube audio: {e}")
        return None

#_______________________

@traceable(name="TranscribeYouTubeAudio")
def transcribe_audio(audio_file_path):
    """
    Transcribes an audio file into text using Whisper.
    """
    try:
        print(f"🎤 Transcribing: {audio_file_path}")
        result = whisper_model.transcribe(audio_file_path)
        transcript_text = result["text"]
        print("✅ Transcription finished.")
        return transcript_text

    except Exception as e:
        print(f"🔥 Error during transcription: {e}")
        return "❌ Error transcribing audio."

#_______________________

@traceable(name="ChunkTranscript")
def chunk_transcript(transcript_text, chunk_size=500):
    """
    Splits a large transcript text into smaller chunks based on sentence boundaries.
    """
    print("✂️ Chunking the transcript...")
    chunks = []
    current_chunk = ""

    for sentence in transcript_text.split('. '):
        if len(current_chunk) + len(sentence) <= chunk_size:
            current_chunk += sentence + ". "
        else:
            chunks.append(current_chunk.strip())
            current_chunk = sentence + ". "

    # Add the last chunk if any remains
    if current_chunk:
        chunks.append(current_chunk.strip())

    print(f"✅ Total chunks created: {len(chunks)}")
    return chunks

# === Embedding Chunks into ChromaDB ===

@traceable(name="EmbedAndStoreChunks")
def embed_and_store_chunks(chunks, youtube_url):
    """
    Embeds the transcript chunks and stores them into ChromaDB.
    """
    print("🧠 Embedding chunks and saving to ChromaDB...")

    # Create IDs for each chunk
    ids = [f"{youtube_url}_chunk_{i}" for i in range(len(chunks))]

    # Embed all chunks
    # embeddings = embedding_model.encode(chunks).tolist()
    embeddings = embedding_model.embed_documents(chunks)

    # Store into Chroma
    video_collection.add(
       documents=chunks,
       embeddings=embeddings,
       ids=ids,
       metadatas=[{"source": youtube_url}] * len(chunks)
)

    print(f"✅ {len(chunks)} chunks embedded and saved successfully.")


# === Create Retriever Function ===

def create_retriever():
    """
    Create a retriever for the 'youtube_videos' collection.
    """
    db = Chroma(
        client=chroma_client,
        collection_name="youtube_videos",
        embedding_function=embedding_model
    )
    retriever = db.as_retriever(search_kwargs={"k": 5})
    return retriever

# === Create QA Chain Function ===

def create_qa_chain():
    """
    Create a RetrievalQA chain using Chroma retriever and OpenAI LLM.
    """
    retriever = create_retriever()
    return RetrievalQA.from_chain_type(
        llm=ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0),
        retriever=retriever,
        return_source_documents=True
    )


# === Handle YouTube Download, Transcription, Chunking, Embedding ===

@traceable(name="HandleYouTubeDownload")
def handle_youtube_download(youtube_url):
    if not youtube_url:
        return "❌ Please enter a valid YouTube link."

    audio_file = download_youtube_audio(youtube_url)
    if audio_file:
        transcript = transcribe_audio(audio_file)

        chunks = chunk_transcript(transcript)

        # === NEW: Embed and Store
        embed_and_store_chunks(chunks, youtube_url)

        global qa_chain
        qa_chain = create_qa_chain()

        # === Preview first 5 chunks
        preview_text = "\n\n".join(chunks[:5])

        return preview_text

    else:
        return "❌ Failed to download audio. Please check the link."

# === QA over Video Function ===

@traceable(name="AnswerVideoQuestion")
def answer_video_question(user_question):
    """
    Answers a user question about the uploaded YouTube video.
    """
    if not user_question:
        return "❌ Please enter a question."

    try:
        print("🔍 Asking video question:", user_question)

        if qa_chain is None:
            print("❌ qa_chain is None")
            return "❌ QA system not initialized."

        result = qa_chain(user_question)
        print("✅ Raw result from QA chain:", result)

        if "result" not in result:
            print("❌ 'result' key missing in QA output.")
            return "❌ Unexpected response format."

        return result["result"]

    except Exception as e:
        print(f"🔥 Error during video QA: {e}")
        return f"❌ Error: {e}"


### ✅ Evaluation / Testing

In [ ]:
# from langchain.evaluation import run_on_dataset
# from langsmith.evaluation import Example, LangChainStringEvaluator
from langchain.evaluation.loading import load_dataset
from langchain.evaluation.schema import StringEvaluator

# === Define AI Judge ===
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI
from langchain.evaluation.criteria.eval_chain import CriteriaEvalChain

# === 1. Define the Nutrition Tool ===
nutrition_tool = Tool(
    name="get_nutrition_info",
    func=get_nutrition_info,
    description="Use this tool to fetch nutrition data for a meal name. Input is the meal name as a string."
)

# === 2. Initialize an LLM for both tools and evaluation ===
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# === 3. Create a LangChain Agent that can use your tool ===
agent = initialize_agent(
    tools=[nutrition_tool],
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

# === 4. Run a test question with the tool ===
print("🔍 Tool Output:")
print(agent.run("What are the nutrition facts for Tomato Omelette?"))

# === 5. Define your LangChain Evaluator (LangSmith compatible) ===
meal_plan_evaluator = CriteriaEvalChain.from_llm(
    llm=llm,
    criteria={
        "relevance": "Does the meal plan match the detected ingredients?",
        "completeness": "Does it cover all 3 main meals (breakfast, lunch, dinner)?",
        "creativity": "Is the meal plan creative and varied?"
    }
)

# === 6. Evaluation Function ===
def evaluate_output(ingredients_list, meal_plan_text):
    try:
        # Format input for evaluator
        inputs = {"ingredients": ", ".join(ingredients_list)}
        prediction = {"meal_plan": meal_plan_text}

        # Run evaluation
        result = meal_plan_evaluator.evaluate_strings(
            input=inputs,
            prediction=prediction
        )

        print("🧠 Judgment:", result["reasoning"])
        print("📊 Score:", result["score"])
        return result

    except Exception as e:
        print(f"🔥 Error during evaluation: {e}")
        return None


In [ ]:
# === Example Evaluation Run ===
ingredients_used = ["cheese", "bread", "tomatoes"]

meal_plan_generated = """
Breakfast: Cheese Omelette
Lunch: Tomato Sandwich
Dinner: Grilled Cheese with Tomato Soup
"""

# Run evaluation
result = evaluate_output(ingredients_used, meal_plan_generated)

## 📍 Step 3: App Layout
The app will have two tabs:
- 📷 Upload Image (Vision)
- 🎤 Record Voice (Speech-to-Text)
Each will detect ingredients, retrieve recipes, plan meals, and show nutrition.

In [ ]:
custom_css = """
body {
    background: linear-gradient(to right, #fdf6e3, #fefcea);
    font-family: 'Segoe UI', sans-serif;
}

.gr-button {
    border-radius: 12px !important;
    font-weight: bold !important;
    padding: 8px 16px !important;
    box-shadow: 1px 1px 3px rgba(0, 0, 0, 0.1);
}

.gr-button:hover {
    background-color: #f4e4c1 !important;
    color: #333 !important;
}

.gr-textbox textarea, .gr-radio label {
    border-radius: 8px !important;
    background-color: #fffefc !important;
}

.gr-chatbot {
    background-color: #fffdfa !important;
    border-radius: 10px;
    border: 1px solid #f0e9dc;
}
"""

with gr.Blocks(css=custom_css) as demo:

    chat_history = gr.State([])
    session_id = gr.State(get_session_id())
    last_dish_state = gr.State("")


    gr.Markdown("""
    <h1 style='text-align: center; color: #5c4b2c;'>🍽️ NutriChef AI</h1>
    <p style='text-align: center; font-size: 17px;'>Your Friendly AI Chef — Generate meal plans, extract nutrition, and answer food questions from image, audio, or video! 🎉</p>
    """)

    with gr.Tab("📷 Upload Fridge Image"):
        with gr.Row():
            with gr.Column():
                image_input = gr.Image(type="pil", label="Upload Fridge Image")
                meal_mode = gr.Radio(
                    choices=["Generate New Meal Plan", "Search Existing Recipes"],
                    label="Choose Task",
                    value="Generate New Meal Plan"
                )
                submit_image = gr.Button("🍳 Find My Meals!", elem_id="btn-image")

            with gr.Column():
                output_text_image = gr.Textbox(label="NutriChef Output", lines=10)

        submit_image.click(
            process_image,
            inputs=[image_input, meal_mode],
            outputs=[output_text_image]
        )

    with gr.Tab("🎤 Upload Ingredients Audio"):
        with gr.Row():
            with gr.Column():
                audio_input = gr.Audio(sources=["upload"], type="filepath", label="Upload Audio (e.g. 'I have eggs and cheese')")
                meal_mode_audio = gr.Radio(
                    choices=["Generate New Meal Plan", "Search Existing Recipes"],
                    label="Choose Task",
                    value="Generate New Meal Plan"
                )
                submit_audio = gr.Button("🎤 Find My Meals!", elem_id="btn-audio")

            with gr.Column():
                output_text_audio = gr.Textbox(label="NutriChef Output", lines=10)

        submit_audio.click(
            process_audio,
            inputs=[audio_input, meal_mode_audio],
            outputs=[output_text_audio]
        )

    with gr.Tab("📹 Upload Cooking Video"):
        with gr.Row():
            with gr.Column():
                youtube_link = gr.Textbox(
                    label="Paste YouTube Link",
                    placeholder="https://www.youtube.com/watch?v=example"
                )
                download_button = gr.Button("🎥 Analyze Video", elem_id="btn-video")

                user_question = gr.Textbox(
                    label="Ask a Question About the Video",
                    placeholder="e.g. What did the chef cook for dinner?"
                )
                ask_button = gr.Button("Ask", elem_id="btn-ask")

            with gr.Column():
                transcript_output = gr.Textbox(
                    label="📜 Transcript Preview",
                    placeholder="Transcript will appear here...",
                    lines=10
                )
                answer_output = gr.Textbox(
                    label="Answer",
                    placeholder="AI response will appear here...",
                    lines=5
                )

        download_button.click(
            handle_youtube_download,
            inputs=[youtube_link],
            outputs=[transcript_output]
        )

        ask_button.click(
            answer_video_question,
            inputs=[user_question],
            outputs=[answer_output]
        )

    ingredients_state = gr.State()
    meal_plan_state = gr.State()

    with gr.Accordion("💬 NutriChef AI Assistant Chat", open=False):
        chatbot = gr.Chatbot(label="NutriChef Chat", value=[])
        user_message = gr.Textbox(label="Your Message", placeholder="Ask me anything...")
        send_button = gr.Button("💬 Send")

        def chat_with_memory(user_message, session_id, chat_history, last_dish):
            try:
                user_message = user_message.strip()
                if not user_message:
                   return gr.update(value=chat_history), "", last_dish

                original_message = user_message  # Keep for dish detection

                # Handle vague follow-ups referring to a previous dish
                vague_phrases = [
                    "how to make it", "how do i make it",
                    "is it healthy", "how much time", "can i add", "should i use"
        ]
                if any(phrase in user_message.lower() for phrase in vague_phrases) and last_dish:
                  llm_input = f"{user_message} (referring to {last_dish})"
                else:
                  llm_input = user_message

                # Retrieve context from Chroma
                recipe_context = get_context_from_query(user_message)

                # Generate response using memory
                response = memory_chain.invoke(
                   {"input": llm_input, "recipe_context": recipe_context},
                   config={"configurable": {"session_id": session_id}}
        )

                # Detect if a new dish is mentioned and store it
                new_dish = detect_dish_with_llm(original_message)
                if new_dish.lower() == "none":
                   new_dish = extract_dish_name(original_message)
                if new_dish and new_dish.lower() != "none":
                   last_dish = new_dish

                # Update chat UI
                chat_history.append([user_message, response.content])
                return gr.update(value=chat_history), "", last_dish

            except Exception as e:
              print(f"🔥 Error in chat_with_memory(): {e}")
              return gr.update(value=chat_history), "", last_dish


        send_button.click(
          chat_with_memory,
          inputs=[user_message, session_id, chat_history, last_dish_state],
          outputs=[chatbot, user_message, last_dish_state]
)


demo.launch() # debug=True
